In [28]:
# !pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
# !pip3 install unstructured
# !pip3 install sentence-transformers

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import JinaEmbeddings


import os

In [2]:
directories = ["Web Scholar PDFs", "Data/About Scottie", "Data/Buggy News", "academic_calendars", "Data/history_of_cmu", "Data/history_of_scs", "Data/Kiltie Band", "Data/lti_faculty", "Data/lti_programs", "program_handbooks", "Data/Tartan Facts"]

In [3]:
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip install pypdf

# !pip3 install ipywidgets


docs = []
for i in directories:
    loader = DirectoryLoader(i, glob="*", exclude="annotation.txt", show_progress=True)
    docs.append(loader.load())

  0%|          | 0/211 [00:00<?, ?it/s]

 83%|████████▎ | 5/6 [00:00<00:00, 202.32it/s]


In [4]:
all_splits = []
for i in docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    all_splits.append(text_splitter.split_documents(i))

In [2]:
import pickle as pkl

# with open('noWebScholarSplitDocuments.pkl', "wb") as f:
#     pkl.dump(all_splits[1:], f)


with open('splitDocuments.pkl','rb') as f: 
  all_splits = pkl.load(f)




In [20]:
for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=GPT4AllEmbeddings(), persist_directory="gpt4allDB/")

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: g

In [24]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=embedding_function, persist_directory="all-MiniLM-L6-v2DB")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
embedding_function = OllamaEmbeddings(model="llama2:7b")

for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=embedding_function, persist_directory="oLlamaDB")

KeyboardInterrupt: 

In [ ]:
embedding_function = OllamaEmbeddings(model="llama2:7b")


for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=embedding_function, persist_directory="oLlamaDB")

In [5]:
# Chroma.from_documents(documents=[all_splits[0][12353]], embedding=GPT4AllEmbeddings(), persist_directory="gpt4allDB/")
embed = GPT4AllEmbeddings()


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [12]:
failed_values = []
for count in range(12350, len(all_splits[0])):
    try:
        res = embed.embed_query(all_splits[0][count].page_content)
        if len(res) == 0:
            print(count)
            failed_values.append(count)
    except Exception as e:
        print(count)
        failed_values.append(count)

12353
12354
12355
12365
12371
12377
12378
12379
12380
12491
12501
12502
12503
12504
12505
15938
15984
20151
20159
20427
20428
20682
22733
22734
22738
22740
22744
23524
23525
23527
23528
23529
23532
23538
23539
23540
23541
23542
23544
23545
23546
23548
23549
23554
23555
28474
28475
28551
28552
28555
28556
28557
28558


In [14]:
paperDocs = all_splits[0]
cleanedPaperDocs = [i for j, i in enumerate(paperDocs) if j not in failed_values]
all_splits[0] = cleanedPaperDocs